# COMPARISION OF EDUCATION LEVELS WITH RESPECT TO EDUCATION

A certain level of degree is required in every field to have more number of jobs in the U.S. market. In some fields employers expect Master's degree or Bachelor's degree or other degrees. We plot the visualization considering employee's education level and the fields they are opting. So,it gives a good insight what level of education is required in major fields. 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
!pip install autocorrect
import numpy as np
import pandas as pd
import os
import seaborn as sns
%matplotlib inline
import clean_wage as cw
import matplotlib.pyplot as plt
import nltk,warnings
from autocorrect import Speller
import gensim.downloader as api
from sklearn.cluster import KMeans
nltk.download('wordnet')
nltk.download('words')

In [0]:
df_1 = pd.read_csv("/content/gdrive/My Drive/H1b_perm/PERM_Disclosure_Data_FY17.csv")
f_1 = df_1[['FOREIGN_WORKER_INFO_MAJOR','FOREIGN_WORKER_INFO_EDUCATION']]
f_1.isna().sum()
f_1.dropna(inplace = True)

In [0]:
df_2 = pd.read_csv('/content/gdrive/My Drive/H1b_perm/PERM_Disclosure_Data_FY15_Q4.csv')
f_2 = df_2[['FOREIGN_WORKER_INFO_MAJOR','FOREIGN_WORKER_INFO_EDUCATION']]
f_2.isna().sum()
f_2.dropna(inplace = True)

In [0]:
df_3 = pd.read_csv('/content/gdrive/My Drive/H1b_perm/PERM_Disclosure_Data_FY16.csv')
f_3 = df_3[['FOREIGN_WORKER_INFO_MAJOR','FOREIGN_WORKER_INFO_EDUCATION']]
f_3.isna().sum()
f_3.dropna(inplace = True)

In [0]:
df_4 = pd.read_csv('/content/gdrive/My Drive/H1b_perm/PERM_Disclosure_Data_FY2018_EOY.csv')
f_4 = df_4[['FOREIGN_WORKER_INFO_MAJOR','FOREIGN_WORKER_INFO_EDUCATION']]
f_4.isna().sum()
f_4.dropna(inplace = True)

In [0]:
perm=pd.concat([f_1,f_2,f_3,f_4])
perm.shape

In [0]:
perm.drop(labels=perm.loc[perm['FOREIGN_WORKER_INFO_EDUCATION']=='None'].index, inplace = True)
perm.drop(labels=perm.loc[perm['FOREIGN_WORKER_INFO_EDUCATION']=='High School'].index, inplace = True)

In [0]:
perm['FOREIGN_WORKER_INFO_EDUCATION'].value_counts()
ax = sns.countplot(x = "FOREIGN_WORKER_INFO_EDUCATION", data = perm)

In [0]:
lemmatizer = nltk.stem.WordNetLemmatizer()
words = set(nltk.corpus.words.words())
spell = Speller()
def lemmatize_text(text):
     return lemmatizer.lemmatize(text)
def spelling_checker(text):
     return spell(text)

In [0]:
perm['FOREIGN_WORKER_INFO_MAJOR']=perm.FOREIGN_WORKER_INFO_MAJOR.apply(lambda txt: " ".join([lemmatize_text(i) for i in txt.lower().split()]))
perm['FOREIGN_WORKER_INFO_MAJOR']=perm.FOREIGN_WORKER_INFO_MAJOR.apply(lambda txt: " ".join([spelling_checker(i) for i in txt.lower().split()]))

In [0]:
perm.reset_index(drop = True,inplace = True)

In [0]:
import string
def remove_punctuation(value):
    result = ""
    for c in value:
        if c not in string.punctuation:
            result += c
        else:
            result +=" "
    return result
perm["FOREIGN_WORKER_INFO_MAJOR"] = perm["FOREIGN_WORKER_INFO_MAJOR"].apply(lambda x : remove_punctuation(x))

In [0]:
perm["FOREIGN_WORKER_INFO_MAJOR"].value_counts()

In [0]:
word_vectors_1 = api.load("glove-wiki-gigaword-100")

In [0]:
def grouping(position):
  # print(position)
    z = np.zeros(100)
    x = position.split()
    count = 0
    for i in x:
        if i in word_vectors_1:
            z += word_vectors_1[i]
            count+=1
    if count>0:
        return z/count
    else:
        return z
perm["word_vectors"] = perm["FOREIGN_WORKER_INFO_MAJOR"].apply(lambda text : grouping(text))

In [0]:
word_vectors = perm[["word_vectors"]].values
temp = np.vstack(word_vectors.tolist())
temp.shape

Using k-means clustering I have grouped them into 20 categories and considered the important categories.

In [0]:
kmeans = KMeans(n_clusters=20, random_state=0).fit(temp)
y = kmeans.predict(temp)

In [0]:
perm.shape

In [0]:
perm["cluster"] = y
perm[perm["cluster"] == 15]

In [0]:
# Assigning names to each category based on their roles after clustering.
perm.loc[(perm.cluster == 1),"cluster"] = "Business Administration"
perm.loc[(perm.cluster == 5),"cluster"] = "Information Technology"
perm.loc[(perm.cluster == 8),"cluster"] = "Information Technology"
perm.loc[(perm.cluster == 2),"cluster"] = "Electrical Engineering"
perm.loc[(perm.cluster == 3),"cluster"] = "Electrical,Electronics and Computer Engineering"
perm.loc[(perm.cluster == 7),"cluster"] = "Computer Science"
perm.loc[(perm.cluster == 10),"cluster"] = "Computer Engineering"
perm.loc[(perm.cluster == 14),"cluster"] = "Computer Science"
perm.loc[(perm.cluster == 17),"cluster"] = "Electronics Engineering"
perm

In [0]:
plot = perm['cluster'].isin(['Information Technology','Electrical,Electronics and Computer Engineering','Business Administration',
                               'Electrical Engineering','Computer Science','Electronics Engineering','Computer Engineering'])
plot_1 = perm[plot]
plot_1.reset_index(drop = True,inplace = True)

In [0]:
x = plot_1["FOREIGN_WORKER_INFO_EDUCATION"].value_counts()
x["Master's"]

In [0]:
bar_plot = plot_1.groupby('cluster')['FOREIGN_WORKER_INFO_EDUCATION'].value_counts()
bar_plot=bar_plot.reset_index(name = 'count')
bar_plot

In [0]:
# Calculating percenage of different levels of education in each categoty how much they are contributing.
for i in bar_plot.index:
    if(bar_plot['FOREIGN_WORKER_INFO_EDUCATION'][i] == "Master's"):
        bar_plot['count'][i] = bar_plot['count'][i]*100/x["Master's"]
        
    elif(bar_plot['FOREIGN_WORKER_INFO_EDUCATION'][i] == "Bachelor's"):
        bar_plot['count'][i] = bar_plot['count'][i]*100/x["Bachelor's"]
        
    elif(bar_plot['FOREIGN_WORKER_INFO_EDUCATION'][i] == "Doctorate"):
        bar_plot['count'][i] = bar_plot['count'][i]*100/x["Doctorate"]
    
    elif(bar_plot['FOREIGN_WORKER_INFO_EDUCATION'][i] == "Other"):
        bar_plot['count'][i] = bar_plot['count'][i]*100/x["Other"]
    
    elif(bar_plot['FOREIGN_WORKER_INFO_EDUCATION'][i] == "Associate's"):
        bar_plot['count'][i] = bar_plot['count'][i]*100/x["Associate's"]
    
    

In [0]:
# Assigning alias names as they are overlapping in the bar plot
bar_plot.loc[(bar_plot.cluster == 'Business Administration'),'cluster'] = 'BA'
bar_plot.loc[(bar_plot.cluster == 'Computer Science'),'cluster'] = 'CS'
bar_plot.loc[(bar_plot.cluster == 'Computer Engineering'),'cluster'] = 'CE'
bar_plot.loc[(bar_plot.cluster == 'Information Technology'),'cluster'] = 'IT'
bar_plot.loc[(bar_plot.cluster == 'Electrical,Electronics and Computer Engineering'),'cluster'] = 'EEC'
bar_plot.loc[(bar_plot.cluster == 'Electrical Engineering'),'cluster'] = 'EE1'
bar_plot.loc[(bar_plot.cluster == 'Electronics Engineering'),'cluster'] = 'EE2'
bar_plot

In [0]:
sns.set(rc={'figure.figsize':(12.0,8.0)})
bar_plot_1 = sns.barplot(x = 'cluster', y = 'count', hue = 'FOREIGN_WORKER_INFO_EDUCATION', palette = "cubehelix_r", data = bar_plot)
leg = bar_plot_1.legend(title = 'Level of Education',loc = (0.7,0.756))
sns.despine(top=True, right=True)
plt.xlabel('Major Degree')
plt.ylabel('Percentage of levels of education(%)')
# plt.yscale("log")
plt.title("Comparision of levels of education")
plt.savefig('Education.png')
plt.show()